# NCEI
<br>

- 월평균 활용
- https://apihub.kma.go.kr/
    - 세계기상 > NCEI 관측.통계 > 3. 전세계 지상관측(월통계) : 1763년~2022년
    - ※ NCEI(National Centers for Environmental Information, 국립환경정보센터)는 미국 NOAA의 소속된 기관으로 전세계 기상기후데이터를 수집·제공합니다.
    - ※ 모든 자료가 2022년 12월 까지 존재합니다.
## 참고 자료
- document 파일
    - 자료 소개서
        - GSOM_readme.txt
    - 자료 설명서
        - GSOM_documentation.pdf
    - 연/월 통합 상세 설명서
        - GSOM_GSOY_Description_Document_v1.0.2_20200219.pdf
- 지점정보
    - 링크 내용은 동일함
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt(내용확인)
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv(파일다운로드)
- 국가 코드
    - Country list
        - 국가명과 국가 2글자 표현 짝 정리된 것
        - 후에 요청인자 중 stn 구성요소인 지점코드값의 국가 부분을 해석하는데에 사용
        - 293개국 존재
- 파일 리스트
    - File list
        - 파일목록
        - 파일명과 파일 크기가 나열됨
        - 후술될 요청인자 중 stn의 구성요소인 지점코드 값으로 활용할 수 있음
            - 국가 2글자, 망구분 1글자, 지점번호 9글자를 가져올 소스
        - 122236개 있음
- 실제 파일 리스트
    - data_climate/gsom-latest 내부에 존재하는 파일들
    - 122041개 있음

## 활용 library

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
dfFiletoCountry = pd.read_csv('data_climate/FiletoCountry.csv')
dfFiletoCountry

,RealFileNM,ID,COUNTRY NAME,Name
0,ACW00011604.csv,AC,ANTIGUA AND BARBUDA,ACW00011604
1,ACW00011647.csv,AC,ANTIGUA AND BARBUDA,ACW00011647
2,AE000041196.csv,AE,NaN,AE000041196
3,AEM00041194.csv,AE,NaN,AEM00041194
4,AEM00041217.csv,AE,NaN,AEM00041217
...,...,...,...,...
122036,ZI000067969.csv,ZI,ZIMBABWE,ZI000067969
122037,ZI000067975.csv,ZI,ZIMBABWE,ZI000067975
122038,ZI000067977.csv,ZI,ZIMBABWE,ZI000067977
122039,ZI000067983.csv,ZI,ZIMBABWE,ZI000067983


## 파일 목록 중에 활용할 csv를 불러오는 함수만들긴
- 필요한 것
    - 적용 국가
    

In [9]:
country = 'Germany'
tmp = dfFiletoCountry[dfFiletoCountry['COUNTRY NAME'].str.contains((country.upper())) == True]
tmp

,RealFileNM,ID,COUNTRY NAME,Name
34657,GM000001153.csv,GM,GERMANY,GM000001153
34658,GM000001474.csv,GM,GERMANY,GM000001474
34659,GM000002277.csv,GM,GERMANY,GM000002277
34660,GM000002288.csv,GM,GERMANY,GM000002288
34661,GM000002698.csv,GM,GERMANY,GM000002698
...,...,...,...,...
35772,GMW00035056.csv,GM,GERMANY,GMW00035056
35773,GMW00035104.csv,GM,GERMANY,GMW00035104
35774,GMW00035108.csv,GM,GERMANY,GMW00035108
35775,GMW00035109.csv,GM,GERMANY,GMW00035109


In [15]:
print(len(tmp))
tmpList = tmp['RealFileNM'].values
print(tmpList)

1120
['GM000001153.csv' 'GM000001474.csv' 'GM000002277.csv' ...
 'GMW00035108.csv' 'GMW00035109.csv' 'GMW00035110.csv']


In [ ]:
def open_csv(country = korea):
    
    # 변수명 리스트
    varList = []
    
    # 
    csvList = dfFiletoCountry[dfFiletoCountrytry['COUNTRY NAME'].str.contains((country.upper())) == True]
    cnt = len(csvList)
    stnList = csvList['RealFileNM'].values
    
    # 퇴근 후 수정하기
    
    for STN in stnList[:10]:
        varNM = 'df_{}'.format(STN)

        globals()[varNM] = pd.read_table('rawdata_climate/' + str(STN) + '.txt', sep = ',',
                                          encoding = 'euc-kr', engine = 'python')
        varList.append(varNM)

        globals()[varNM].to_csv('data_climate/' + str(STN) + '.csv', index = False)

    # 변수명 리스트
    print(varList)

    # 변수명 리스트에서 str 값을 변수로 활용하는 방법
#     globals()[varList[0]]
    
    return varList

In [ ]:
stnList = []
for idx, row in dfFileList.iterrows():
    tmp = row[0][5:-4]
    stnList.append(tmp)
print(len(stnList))
print(stnList[:5])

In [ ]:
def location_collect(num1 = None, num2 = None):
    path = 'data_climate/gsom-latest/'
    flag = False

    for file in file_list_csv[num1: num2]:

        # 활용할 csv
        dfTMP = pd.read_csv(path + file)
#         print('1')
        # csv 처음 불러왔을 때
        if flag == False:
            print('0')
            # 결과 df 변수에 현재의 첫 번째 행을 넣는다.
            df = dfTMP.loc[0:0, :]
            # flag를 변경하여 이 if문에 다시 들어오지 않는다.
            flag = True
            # 확인용 출력
#             print('길이: ', len(df))
        # csv 처음 불러온 것이 아닐 때
        else:
#             print('2')
            # 결과 df와 지금 df의 첫 번째 행을 이어붙인다.
            df = pd.concat([df, dfTMP.loc[0:0, :]], axis = 0)
#             print('길이: ', len(df))
#             print('3')
    df.reset_index(inplace = True, drop = True)
    print(len(df))
    return df

In [ ]:
# txt > csv
for STN in stnList[:10]:

varList = []
for STN in stnList[:10]:
    varNM = 'df_{}'.format(STN)
    
    globals()[varNM] = pd.read_table('rawdata_climate/' + str(STN) + '.txt', sep = ',',
                                      encoding = 'euc-kr', engine = 'python')
    varList.append(varNM)
    
    globals()[varNM].to_csv('data_climate/' + str(STN) + '.csv', index = False)

# 변수명 리스트
print(varList)

# 변수명 리스트에서 str 값을 변수로 활용하는 방법
globals()[varList[0]]